<h1 align = center>Segmenting and Clustering Neighborhood in Toronto</h1>

<h2>Introduction</h2>
<p>In this assignment, I used files from previous assginments that convert location to latitude and longitude. Then I used Foursquare API <b>explore</b> function to search for the most commen venue cotegories in each neighborhood and then uses this feature to group the neighborhood into three clusters. The algorithm I used to cluster the dataset is <b>K-means</b>. To evaluate and present the model, I used <b>folium</b> as the primary visualization tool and exam top ten commen venues in each cluster to gain insights.
</p>

<h2>Import libraries and file</h2>

In [1]:
import pandas as pd
import numpy as np
import folium
from geopy import Nominatim
import requests
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
df = pd.read_csv('postal_CAN_LATLON.csv').drop(['Unnamed: 0'], axis=1)

In [3]:
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [4]:
df['Borough'].value_counts()

North York            24
Downtown Toronto      18
Scarborough           17
Etobicoke             11
Central Toronto        9
West Toronto           6
East Toronto           5
York                   5
East York              5
EtobicokeNorthwest     1
Mississauga            1
Name: Borough, dtype: int64

#### Aggregate Downtown Toronto, Central Toronto, West Toronto and East Toronto for this assignment

In [5]:
toronto_data = df[(df['Borough'] == 'Downtown Toronto')|(df['Borough'] == 'Central Toronto')
                 |(df['Borough'] == 'West Toronto')|(df['Borough'] == 'East Toronto')].reset_index(drop=True)
toronto_data['Borough'].value_counts()

Downtown Toronto    18
Central Toronto      9
West Toronto         6
East Toronto         5
Name: Borough, dtype: int64

#### Convert address to coordinates 

In [8]:
address = 'Toronto'
geolocator = Nominatim(user_agent='Ont_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

#this latitude and longitude will be used in the next cell to plot a map
print("The location for plotting the map: latitude = {}, longitude = {} ".format(latitude,longitude))

The location for plotting the map: latitude = 43.653963, longitude = -79.387207 


#### Plot data on the map using folium 

I want to plot a map using the latitude and longitude converted from above cell with the starting zoom level to be 11. I will use a circle marker on the map with blue color boundary, radius 5, fill makers with #3186cc and opacity of 70%. Also, set popup labels to be Neighborhood names.

In [9]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lon, label in zip(df['Latitude'],df['Longitude'],df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lon],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=True).add_to(map_toronto)
map_toronto

#### Make API calls to Foursquare to get top 100 commen venues
Set my Foursquare credential variables

In [11]:
CLIENT_ID = 'My credentials' # your Foursquare ID
CLIENT_SECRET = 'My credentials' # your Foursquare Secret
VERSION = 'Date of Today' # Foursquare API version
LIMIT = 100 #Search the first 100 commen veneus

Define the function to loop through and get all venues 

In [12]:
def get_venues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lon in zip(names, latitudes, longitudes):
        #generate the link for the api call
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lon,
            radius,
            LIMIT)
        
        #get the json file from each call
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        #process the json file to get the needed information
        venues_list.append([(
                name, 
                lat, 
                lon, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])
    
    #use imbeded loops to write all information to the dataset
    nearby_venues = pd.DataFrame([item for venue in venues_list for item in venue])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
#call the function
toronto_venues = get_venues(names=df['Neighborhood'],
                            latitudes=df['Latitude'],
                            longitudes=df['Longitude'])

In [130]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa


In [17]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",12,12,12,12,12,12
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
...,...,...,...,...,...,...
WillowdaleWest,6,6,6,6,6,6
Woburn,3,3,3,3,3,3
Woodbine Heights,9,9,9,9,9,9


#### Prepare the dataframe 
Use one hot encoding to convert categorical value to dummy numercical variable

In [15]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood, Neighborhood Latitude and Neighborhood Longitude column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
toronto_onehot['Neighborhood Latitude'] = toronto_venues['Neighborhood Latitude']
toronto_onehot['Neighborhood Longitude'] = toronto_venues['Neighborhood Longitude']

# move neighborhood, Neighborhood Latitude and Neighborhood Longitude to the front of the dataframe
toronto_onehot = toronto_onehot[['Neighborhood Longitude']+[col for col in toronto_onehot.columns if col != 'Neighborhood Longitude']]
toronto_onehot = toronto_onehot[['Neighborhood Latitude']+[col for col in toronto_onehot.columns if col != 'Neighborhood Latitude']]
toronto_onehot = toronto_onehot[['Neighborhood']+[col for col in toronto_onehot.columns if col != 'Neighborhood']]
toronto_onehot.dropna(axis=0,inplace=True)

In [16]:
toronto_onehot.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Malvern, Rouge",43.806686,-79.194353,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Standardized data using mean frequency of occurence of each category per group of neighborhood

In [18]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,43.794200,-79.262029,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",43.605647,-79.501321,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,43.786947,-79.385975,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",43.733282,-79.419750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,WillowdaleWest,43.782736,-79.442259,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
94,Woburn,43.770992,-79.216917,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95,Woodbine Heights,43.695344,-79.318389,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,York Mills West,43.752758,-79.400049,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### create another dataframe containing the top 10 venues for cluster evaluation 

In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[2:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [20]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        #this line of code only works before OutOfBoundary exception happens
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        #this line of code works as the exception handler
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)
    
# move Neighborhood Latitude and Neighborhood longitude to the front of the dataframe
neighborhoods_venues_sorted.insert(1, 'Neighborhood Latitude', toronto_grouped['Neighborhood Latitude'])
neighborhoods_venues_sorted.insert(2, 'Neighborhood Longitude', toronto_grouped['Neighborhood Longitude'])
neighborhoods_venues_sorted.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,43.794200,-79.262029,Lounge,Latin American Restaurant,Skating Rink,Clothing Store,Breakfast Spot,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run
1,"Alderwood, Long Branch",43.605647,-79.501321,Fried Chicken Joint,American Restaurant,Seafood Restaurant,Fast Food Restaurant,Café,Bakery,Restaurant,Liquor Store,Pharmacy,Gym
2,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,Coffee Shop,Bank,Pharmacy,Frozen Yogurt Shop,Bridal Shop,Sandwich Place,Diner,Restaurant,Middle Eastern Restaurant,Supermarket
3,Bayview Village,43.786947,-79.385975,Café,Bank,Chinese Restaurant,Japanese Restaurant,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
4,"Bedford Park, Lawrence Manor East",43.733282,-79.419750,Italian Restaurant,Coffee Shop,Sandwich Place,Restaurant,Thai Restaurant,Juice Bar,Fast Food Restaurant,Butcher,Indian Restaurant,Sushi Restaurant


#### Use Kmeans to cluster the neighborhoods 

In [21]:
k = 3 #set number of clusters

#drop unwanted features 
toronto_grouped_clustering = toronto_grouped.drop(['Neighborhood','Neighborhood Latitude','Neighborhood Longitude'], axis=1)

#perform Kmeans and check lables for the first 20 items
kmeans = KMeans(n_clusters = k, random_state=1).fit(toronto_grouped_clustering)
kmeans.labels_[0:20]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [22]:
#insert cluster labels to the cluter evaluation dataframe
neighborhoods_venues_sorted.insert(1,'Cluster Labels', kmeans.labels_)

In [23]:
neighborhoods_venues_sorted

,Neighborhood,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,1,43.794200,-79.262029,Lounge,Latin American Restaurant,Skating Rink,Clothing Store,Breakfast Spot,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run
1,"Alderwood, Long Branch",1,43.605647,-79.501321,Fried Chicken Joint,American Restaurant,Seafood Restaurant,Fast Food Restaurant,Café,Bakery,Restaurant,Liquor Store,Pharmacy,Gym
2,"Bathurst Manor, Wilson Heights, Downsview North",1,43.754328,-79.442259,Coffee Shop,Bank,Pharmacy,Frozen Yogurt Shop,Bridal Shop,Sandwich Place,Diner,Restaurant,Middle Eastern Restaurant,Supermarket
3,Bayview Village,1,43.786947,-79.385975,Café,Bank,Chinese Restaurant,Japanese Restaurant,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
4,"Bedford Park, Lawrence Manor East",1,43.733282,-79.419750,Italian Restaurant,Coffee Shop,Sandwich Place,Restaurant,Thai Restaurant,Juice Bar,Fast Food Restaurant,Butcher,Indian Restaurant,Sushi Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,WillowdaleWest,1,43.782736,-79.442259,Pizza Place,Coffee Shop,Bank,Discount Store,Butcher,Pharmacy,Comic Shop,Dim Sum Restaurant,Empanada Restaurant,Electronics Store
94,Woburn,1,43.770992,-79.216917,Coffee Shop,Korean Restaurant,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Dumpling Restaurant
95,Woodbine Heights,1,43.695344,-79.318389,Spa,Beer Store,Park,Cosmetics Shop,Curling Ice,Diner,Skating Rink,Pharmacy,Video Store,General Travel
96,York Mills West,0,43.752758,-79.400049,Park,Bank,Convenience Store,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant


#### Visualize the  results

In [24]:
#similar settings as the first map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

#create colors for each cluster
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#create the plot
for lat, lon, poi, cluster in zip(neighborhoods_venues_sorted['Neighborhood Latitude'], neighborhoods_venues_sorted['Neighborhood Longitude'],
                                 neighborhoods_venues_sorted['Neighborhood'], neighborhoods_venues_sorted['Cluster Labels']):
    label = folium.Popup(str(poi) + '\n' + 'Cluster' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

#### Evaluated the three clusters 

cluster 1 is more friendly to people likes outdoor activities and asian food

In [193]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 0]

,Neighborhood,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Caledonia-Fairbanks,0,43.689026,-79.453512,Park,Women's Store,Market,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
25,DownsviewEast,0,43.737473,-79.464763,Airport,Park,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
55,"Milliken, Agincourt North, Steeles East, L'Amo...",0,43.815252,-79.284577,Playground,Park,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
58,"Moore Park, Summerhill East",0,43.689574,-79.383160,Park,Trail,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
66,Parkwoods,0,43.753259,-79.329656,Park,Food & Drink Shop,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
69,Rosedale,0,43.679563,-79.377529,Park,Playground,Trail,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
83,The Danforth East,0,43.685347,-79.338106,Convenience Store,Metro Station,Park,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio
96,York Mills West,0,43.752758,-79.400049,Park,Bank,Convenience Store,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant


Cluster 2 is more friendly to food lovers 

In [194]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 1]

,Neighborhood,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,1,43.794200,-79.262029,Lounge,Latin American Restaurant,Skating Rink,Clothing Store,Breakfast Spot,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run
1,"Alderwood, Long Branch",1,43.605647,-79.501321,Fried Chicken Joint,American Restaurant,Seafood Restaurant,Fast Food Restaurant,Café,Bakery,Restaurant,Liquor Store,Pharmacy,Gym
2,"Bathurst Manor, Wilson Heights, Downsview North",1,43.754328,-79.442259,Coffee Shop,Bank,Pharmacy,Frozen Yogurt Shop,Bridal Shop,Sandwich Place,Diner,Restaurant,Middle Eastern Restaurant,Supermarket
3,Bayview Village,1,43.786947,-79.385975,Café,Bank,Chinese Restaurant,Japanese Restaurant,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
4,"Bedford Park, Lawrence Manor East",1,43.733282,-79.419750,Italian Restaurant,Coffee Shop,Sandwich Place,Restaurant,Thai Restaurant,Juice Bar,Fast Food Restaurant,Butcher,Indian Restaurant,Sushi Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,WillowdaleSouth,1,43.770120,-79.408493,Ramen Restaurant,Café,Sandwich Place,Coffee Shop,Sushi Restaurant,Restaurant,Pizza Place,Ice Cream Shop,Fast Food Restaurant,Hotel
93,WillowdaleWest,1,43.782736,-79.442259,Pizza Place,Coffee Shop,Bank,Discount Store,Butcher,Pharmacy,Comic Shop,Dim Sum Restaurant,Empanada Restaurant,Electronics Store
94,Woburn,1,43.770992,-79.216917,Coffee Shop,Korean Restaurant,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Dumpling Restaurant
95,Woodbine Heights,1,43.695344,-79.318389,Spa,Beer Store,Park,Cosmetics Shop,Curling Ice,Diner,Skating Rink,Pharmacy,Video Store,General Travel


Cluster 3 seems to be similar to cluster 1

In [195]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster Labels'] == 2]

,Neighborhood,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,"Old Mill South, King's Mill Park, Sunnylea, Hu...",2,43.653654,-79.506944,River,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Department Store
